# Diabetes Risk Prediction
This project uses the Behavioral Risk Factor Surveillance System (BRFSS) survey data from [this link](https://www.cdc.gov/brfss/annual_data/annual_2024.html) to predict the probability of developing different types of Diabetes. Features about U.S. residents include demographic data (e.g. income level, education, race) as well as data regarding health-related risk behaviors, chronic health conditions, and use of preventive services.

This is the first notebook for the project, which parses the raw ASCII data file, available in the link above, to extract the relevant target and feature variables for subsequent EDA and modeling.

## Setup
### Define parameters

In [22]:
raw_data_url = "https://www.cdc.gov/brfss/annual_data/2024/files/LLCP2024ASC.zip"
data_dict_url = "https://www.cdc.gov/brfss/annual_data/2024/zip/codebook24_llcp-v2-508.zip"

### Import packages

In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import re

## Download data
### Raw data ASCII file

In [5]:
!wget {raw_data_url}

--2025-11-04 23:15:44--  https://www.cdc.gov/brfss/annual_data/2024/files/LLCP2024ASC.zip
Resolving www.cdc.gov (www.cdc.gov)... 23.6.96.221, 2600:1409:9800:1a88::2461, 2600:1409:9800:1a82::2461
Connecting to www.cdc.gov (www.cdc.gov)|23.6.96.221|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 53411907 (51M) [application/x-zip-compressed]
Saving to: ‘LLCP2024ASC.zip.1’

LLCP2024ASC.zip.1   100%[===================>]  50.94M  42.9MB/s    in 1.2s    

2025-11-04 23:15:45 (42.9 MB/s) - ‘LLCP2024ASC.zip.1’ saved [53411907/53411907]



In [6]:
asc_zip = !ls LLCP2024ASC*.zip

Archive:  LLCP2024ASC.zip
  inflating: LLCP2024.ASC            


In [20]:
asc_file = !ls LLCP2024*.ASC*
asc_file

['LLCP2024.ASC ']

### Data dictionary file (HTML)

In [23]:
!wget {data_dict_url}

--2025-11-04 23:39:22--  https://www.cdc.gov/brfss/annual_data/2024/zip/codebook24_llcp-v2-508.zip
Resolving www.cdc.gov (www.cdc.gov)... 23.6.96.221, 2600:1409:9800:1a82::2461, 2600:1409:9800:1a88::2461
Connecting to www.cdc.gov (www.cdc.gov)|23.6.96.221|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 71439 (70K) [application/x-zip-compressed]
Saving to: ‘codebook24_llcp-v2-508.zip’

codebook24_llcp-v2- 100%[===================>]  69.76K  --.-KB/s    in 0.02s   

2025-11-04 23:39:27 (3.56 MB/s) - ‘codebook24_llcp-v2-508.zip’ saved [71439/71439]



In [24]:
dict_zip = !ls codebook24_llcp*.zip

In [26]:
!unzip {dict_zip[0]}

Archive:  codebook24_llcp-v2-508.zip
  inflating: USCODE24_LLCP_082125.HTML  


## Parse data

In [ ]:
def build_column_lookup(html_file):
    """
    Parse HTML data dictionary to extract column definitions.
    
    Parameters:
    -----------
    html_file : str
        Path to the HTML data dictionary file
    
    Returns:
    --------
    dict : Dictionary mapping variable labels to their metadata
           Format: {label: {'column_range': str, 'type': str, 'sas_name': str}}
    """
    with open(html_file, 'r', encoding='windows-1252') as f:
        soup = BeautifulSoup(f, 'html.parser')
    
    column_lookup = {}
    
    # Find all variable definition cells in table headers
    for cell in soup.find_all('td', {'class': 'l m linecontent'}):
        text = cell.get_text()
        
        # Only process cells that contain variable definitions
        if 'Label:' in text and 'Column:' in text:
            # Extract label (between "Label:" and next section)
            label_match = re.search(r'Label:\s*(.+?)(?:\n|Section)', text)
            
            # Extract column range (format: "N" or "N-M")
            column_match = re.search(r'Column:\s*(\d+(?:-\d+)?)', text)
            
            # Extract variable type (Num or Char)
            type_match = re.search(r'Type of Variable:\s*(Num|Char)', text)
            
            # Extract SAS variable name
            varname_match = re.search(r'SAS Variable Name:\s*(\w+)', text)
            
            if label_match and column_match:
                label = label_match.group(1).strip()
                column_range = column_match.group(1)
                var_type = type_match.group(1) if type_match else None
                var_name = varname_match.group(1) if varname_match else None
                
                column_lookup[label] = {
                    'column_range': column_range,
                    'type': var_type,
                    'sas_name': var_name
                }
    
    print(f"Successfully parsed {len(column_lookup)} variable definitions from HTML dictionary")
    return column_lookup

In [ ]:
# Step 1: Define columns you want to extract (using labels from HTML)
# To add more columns, simply add their labels to this list
columns_to_extract = [
    "State FIPS Code",
    "Annual Sequence Number",
    # Add more labels here as needed (up to 25 total)
]

# Step 2: Build lookup dictionary from HTML data dictionary
html_file = 'USCODE24_LLCP_082125.HTML'
column_lookup = build_column_lookup(html_file)

# Step 3: Convert labels to colspecs for pd.read_fwf()
colspecs = []
column_names = []
dtypes = {}

for label in columns_to_extract:
    if label in column_lookup:
        col_info = column_lookup[label]
        col_range = col_info['column_range']
        
        # Parse "1-2" or "149" format
        if '-' in col_range:
            start, end = map(int, col_range.split('-'))
        else:
            start = end = int(col_range)
        
        # Convert to 0-based indexing for Python
        colspecs.append((start - 1, end))
        
        # Use SAS variable name for column name
        col_name = col_info['sas_name']
        column_names.append(col_name)
        
        # Set dtype (start with string for safety, can convert later)
        dtypes[col_name] = str
        
        print(f"  {label} -> {col_name} (columns {col_range})")
    else:
        print(f"  WARNING: '{label}' not found in data dictionary")

print(f"\nPrepared to extract {len(colspecs)} columns from ASCII file")

# Step 4: Read the ASCII file using pd.read_fwf()
print(f"\nReading ASCII file: {asc_file[0]}")
df = pd.read_fwf(
    asc_file[0], 
    colspecs=colspecs,
    names=column_names,
    dtype=dtypes,
    encoding='ascii'
)

print(f"Successfully loaded {len(df):,} rows and {len(df.columns)} columns")

# Step 5: Display results
print("\nFirst 5 rows:")
display(df.head())

print("\nDataset info:")
print(df.info())

print("\nColumn value counts:")
for col in df.columns:
    print(f"\n{col}:")
    print(df[col].value_counts().head(10))